<a href="https://colab.research.google.com/github/coll-j/IndonesianDepParse/blob/master/gegem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [156]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence, pack_sequence, PackedSequence

In [366]:
a = torch.rand(2, 1, 5)
print("before")
print(a)
a = a.unsqueeze_(-1).expand(2, 1, 5, 5)
print("after")
print(a)

before
tensor([[[0.5824, 0.7257, 0.9338, 0.1407, 0.9204]],

        [[0.2018, 0.5768, 0.7079, 0.4656, 0.3159]]])
after
tensor([[[[0.5824, 0.5824, 0.5824, 0.5824, 0.5824],
          [0.7257, 0.7257, 0.7257, 0.7257, 0.7257],
          [0.9338, 0.9338, 0.9338, 0.9338, 0.9338],
          [0.1407, 0.1407, 0.1407, 0.1407, 0.1407],
          [0.9204, 0.9204, 0.9204, 0.9204, 0.9204]]],


        [[[0.2018, 0.2018, 0.2018, 0.2018, 0.2018],
          [0.5768, 0.5768, 0.5768, 0.5768, 0.5768],
          [0.7079, 0.7079, 0.7079, 0.7079, 0.7079],
          [0.4656, 0.4656, 0.4656, 0.4656, 0.4656],
          [0.3159, 0.3159, 0.3159, 0.3159, 0.3159]]]])


In [2]:
%%bash
rm test.txt
wget https://raw.githubusercontent.com/coll-j/IndonesianDepParse/master/test.txt

rm: cannot remove 'test.txt': No such file or directory
--2020-09-02 07:17:10--  https://raw.githubusercontent.com/coll-j/IndonesianDepParse/master/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 316 [text/plain]
Saving to: ‘test.txt’

     0K                                                       100% 13.2M=0s

2020-09-02 07:17:10 (13.2 MB/s) - ‘test.txt’ saved [316/316]



In [ ]:
pip install nlp-id
import nltk
nltk.download('punkt')

from nlp_id.tokenizer import Tokenizer
from nlp_id.postag import PosTag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [174]:
args = {
    'batch_size': 1,
    'word_emb_dim': 100,
    'pos_emb_dim': 100,
    'dep_emb_dim': 100,
    'hidden_dim': 128,
    'num_layers': 20,
    'dropout': 0.33,
    'dba_hidden_dim': 128,
    'learning_rate': 0.0001,
    'clip': 0.25,
    'log_interval': 1
}
FIELD_TO_IDX = {'idx': 0, 'word': 1, 'postag': 2, 'head': 3, 'deprel': 4}

In [5]:
from sklearn import preprocessing
def get_long_tensor(tokens_list, batch_size, vocabs=None):
    """ Convert (list of )+ tokens to a padded LongTensor. """
    # sizes = []
    # for tokens_list in batch:
    #   x = tokens_list
    #   sizes.append(max(len(y) for y in x))
    #   tokens = torch.LongTensor(batch_size, *sizes).fill_(0)
    #   for i, s in enumerate(tokens_list):
    #     # ten = torch.LongTensor(list(map(ord, s[:len(s)])))
    #     if vocabs is not None:
    #       ten = torch.LongTensor([vocabs[s]])
    #     else:
    #       ten = torch.LongTensor([int(s)])
    #     tokens[i, :len(s)] = ten
      
    # return tokens
    sizes = []
    x = tokens_list
    while isinstance(x[0], list):
        sizes.append(max(len(y) for y in x))
        x = [z for y in x for z in y]
    tokens = torch.LongTensor(batch_size, *sizes).fill_(0)
    for i, s in enumerate(tokens_list):
        tokens[i, :len(s)] = torch.LongTensor(s)
    return tokens

In [6]:
class Loader:
  
  def __init__(self, file_path, batch_size):
    self._file_path = file_path
    self.batch_size = batch_size

    self.data = self.load_file()
    self.vocabs = self.get_vocabs(self.data)
    self.data = self.preprocess(self.data, self.vocabs)
    self.data = self.chunk_batches(self.data)

  def load_file(self):
    sents, sent = [], []
    f = open(self._file_path)
    for line in f:
      line = line.strip()
      if len(line) == 0:
        if len(sent) > 0:
          sents.append(sent)
          sent = []

      else:
        cols = line.split('\t')
        sent += [cols]
    if len(sent) > 0:
      sents.append(sent)

    return sents

  def get_vocabs(self, data):
    words = [cols[FIELD_TO_IDX['word']] for sent in data for cols in sent]
    postags = [cols[FIELD_TO_IDX['postag']] for sent in data for cols in sent] 
    deprels = [cols[FIELD_TO_IDX['deprel']] for sent in data for cols in sent] 

    words = self.build_vocab(words)
    postags = self.build_vocab(postags)
    deprels = self.build_vocab(deprels)

    return {'words': words, 'postags': postags, 'deprels': deprels}

  def build_vocab(self, cols):
    w2i = {}
    for w in cols:
      if w not in w2i:
        w2i[w] = len(w2i)
    
    return w2i

  def preprocess(self, data, vocabs):
    processed = []
    for sent in data:
      p_sent = [[vocabs['words'][w[FIELD_TO_IDX['word']]] for w in sent]]
      p_sent += [[vocabs['postags'][w[FIELD_TO_IDX['postag']]] for w in sent]]
      p_sent += [[int(w[FIELD_TO_IDX['head']]) for w in sent]]
      p_sent += [[vocabs['deprels'][w[FIELD_TO_IDX['deprel']]] for w in sent]]
      processed.append(p_sent)

    return processed

  def chunk_batches(self, data):
    res, curr = [], []
    currlen = 0
    for sent in data:
      if len(curr) >= self.batch_size:
        res.append(curr)
        curr = []
        currlen = 0
      
      curr.append(sent)
      currlen += len(sent[0])
    if len(curr) >= self.batch_size:
      res.append(curr)

    return res
    
  def __getitem__(self, key):
    batch = self.data[key]
    batch = list(zip(*batch))
    
    # convert to tensors
    words = batch[0]
    words = get_long_tensor(words, len(words), vocabs=self.vocabs['words'])
    words_mask = torch.eq(words, 0)

    postags = batch[1]
    postags = get_long_tensor(postags, len(postags), vocabs=self.vocabs['postags'])
    
    heads = batch[2]
    heads = get_long_tensor(heads, len(heads))
    
    deprels = batch[3]
    deprels = get_long_tensor(deprels, len(deprels), vocabs=self.vocabs['deprels'])
    sentlens = [len(sent) for sent in batch[0]]

    return words, words_mask, postags, heads, deprels, sentlens

  def __iter__(self):
    for i in range(len(self.data)):
      yield self.__getitem__(i)

  def __len__(self):
    return len(self.data)

In [134]:
data = Loader('test.txt', args['batch_size'])

In [92]:
data.vocabs

{'deprels': {'advmod': 2,
  'appos': 6,
  'nmod': 5,
  'nsubj': 3,
  'nummod': 0,
  'obj': 1,
  'punct': 7,
  'root': 4},
 'postags': {'ADV': 2, 'IN': 5, 'NN': 1, 'NUM': 0, 'PR': 3, 'SYM': 6, 'VB': 4},
 'words': {'.': 14,
  'Banyak': 0,
  'Buku': 5,
  'Kita': 8,
  'baca': 4,
  'berada': 10,
  'buku': 1,
  'dalam': 12,
  'di': 11,
  'dibaca': 7,
  'ini': 6,
  'ruangan': 13,
  'saya': 3,
  'sedang': 9,
  'telah': 2}}

In [ ]:
data.data

# Build Model

In [8]:
class DeepBiaffine(nn.Module):
  def __init__(self, input1_size, input2_size, hidden_size, output_size, dropout=0.0):
    super(DeepBiaffine, self).__init__()
    # Simple MLP
    self.W1 = nn.Linear(input1_size, hidden_size)
    self.W2 = nn.Linear(input2_size, hidden_size)
    self.relu = F.relu
    
    # Biaffine
    self.biaff = nn.Bilinear(hidden_size + 1, hidden_size + 1, output_size)

    self.drop = nn.Dropout(dropout)

  def forward(self, input1, input2):
    output1 = self.drop(self.relu(self.W1(input1)))
    output2 = self.drop(self.relu(self.W2(input2)))
    output1 = torch.cat([output1, output1.new_ones(*output1.size()[:-1], 1)], len(output1.size())-1)
    output2 = torch.cat([output2, output2.new_ones(*output2.size()[:-1], 1)], len(output2.size())-1)
    # print(output1.shape)
    # output1 = output1.unsqueeze(len(output1.size())-1).expand(output1.size(0), output1.size(1), output1.size(1), output1.size(2))
    # output2 = output2.unsqueeze(len(output2.size())-1).expand(output2.size(0), output2.size(1), output2.size(1), output2.size(2))

    # print(output1.shape)

    # return
    return self.biaff(output1, output2)

In [210]:
class Model(nn.Module):
  def __init__(self, args, vocab):
    super(Model, self).__init__()

    self.args = args
    self.vocab = vocab

    # input layer
    input_size = 0
    self.word_emb = nn.Embedding(len(vocab['words']), self.args['word_emb_dim'])
    input_size += self.args['word_emb_dim']
    self.pos_emb = nn.Embedding(len(vocab['postags']), self.args['pos_emb_dim'])
    input_size += self.args['pos_emb_dim']
    self.dep_emb = nn.Embedding(len(vocab['deprels']), self.args['dep_emb_dim'])

    # recurrent layer
    self.GRU = nn.GRU(input_size, self.args['hidden_dim'], self.args['num_layers'],\
                            batch_first=True, dropout=self.args['dropout'], bidirectional=True)
    self.GRU_hidden = nn.Parameter(torch.zeros(self.args['num_layers'] * 2, self.args['batch_size'], self.args['hidden_dim']))
    
    # classifier
    self.unlabeled = DeepBiaffine(self.args['hidden_dim'] * 2, self.args['hidden_dim'] * 2, self.args['dba_hidden_dim'], 1, dropout=self.args['dropout'])
    self.deprel = DeepBiaffine(self.args['hidden_dim'] * 2, self.args['hidden_dim'] * 2, self.args['dba_hidden_dim'], len(vocab['deprels']), dropout=self.args['dropout'])

    # criterion
    self.crit = nn.CrossEntropyLoss(ignore_index=-1)
    
    self.dropout = nn.Dropout(self.args['dropout'])
  def forward(self, words, word_mask, postags, heads, deprels, sentlens):
    #TODO: make forward func
    inputs = []

    # pack embedded inputs
    embedded_word = self.word_emb(words)
    inputs += [pack_padded_sequence(embedded_word, sentlens, batch_first=True)]

    embedded_pos = self.pos_emb(postags)
    inputs += [pack_padded_sequence(embedded_pos, sentlens, batch_first=True)]

    # rnn inputs
    rnn_inputs = torch.cat([x.data for x in inputs], 1)
    rnn_inputs = self.dropout(rnn_inputs)
    rnn_inputs = PackedSequence(rnn_inputs, inputs[0].batch_sizes)

    rnn_outputs, _ = self.GRU(rnn_inputs, self.GRU_hidden)
    rnn_outputs, _ = pad_packed_sequence(rnn_outputs, batch_first=True)
    # print("rnn ", rnn_outputs.shape)
    unlabeled_inputs = rnn_outputs.unsqueeze(len(rnn_outputs.size())-1).expand(rnn_outputs.size(0), rnn_outputs.size(1), rnn_outputs.size(1), rnn_outputs.size(2))
    # print("rnn1 ", rnn_outputs.shape)

    #TODO: classify outputs, calc loss
    unlabeled_scores = self.unlabeled(self.dropout(unlabeled_inputs), self.dropout(unlabeled_inputs))
    deprel_scores = self.deprel(self.dropout(rnn_outputs), self.dropout(rnn_outputs))
    
    # unlabeled_scores = unlabeled_scores[:, 1:, :]
    unlabeled_scores = unlabeled_scores.contiguous().view(-1, unlabeled_scores.size(2))
    deprel_scores = deprel_scores.contiguous().view(-1, len(self.vocab['deprels']))

    heads -= 1
    preds = []
    if self.training:
      loss = self.crit(unlabeled_scores, heads.view(-1))
      loss += self.crit(deprel_scores, deprels.view(-1))
      
      # calculate accuracy
      unlabeled_preds = F.log_softmax(unlabeled_scores, 1)
      unlabeled_preds = unlabeled_preds.argmax(dim=1)
      ul_corr = (unlabeled_preds == heads.view(-1))
      acc1 = ul_corr.sum().float() / float( heads.view(-1).size(0) )

      deprel_preds = F.log_softmax(deprel_scores, 1)
      deprel_preds = deprel_scores.argmax(dim=1)
      dep_corr = (deprel_preds == deprels.view(-1))
      acc2 = dep_corr.sum().float() / float( deprels.view(-1).size(0) )

      acc = (acc1 + acc2)/2
    else:
      loss = self.crit(unlabeled_scores, heads.view(-1))
      loss += self.crit(deprel_scores, deprels.view(-1))

      # calculate accuracy
      unlabeled_preds = F.log_softmax(unlabeled_scores, 1)
      unlabeled_preds = unlabeled_preds.argmax(dim=1)
      ul_corr = (unlabeled_preds == heads.view(-1))
      acc1 = ul_corr.sum().float() / float( heads.view(-1).size(0) )

      deprel_preds = F.log_softmax(deprel_scores, 1)
      deprel_preds = deprel_scores.argmax(dim=1)
      dep_corr = (deprel_preds == deprels.view(-1))
      acc2 = dep_corr.sum().float() / float( deprels.view(-1).size(0) )

      acc = (acc1 + acc2)/2

      # predictions
      unlabeled_preds = F.log_softmax(unlabeled_scores, 1)
      unlabeled_preds = unlabeled_preds.argmax(dim=1)
      unlabeled_preds += 1

      deprel_preds = F.log_softmax(deprel_scores, 1)
      deprel_preds = deprel_scores.argmax(dim=1)

      preds.append(unlabeled_preds.detach().cpu().numpy())
      preds.append(deprel_preds.detach().cpu().numpy())
    return loss, acc, preds

In [211]:
model = Model(args, data.vocabs)

# Train Function

In [218]:
import time
def train_model(args, model, train_data, eval_data, num_epochs):
  optimizer = optim.Adam(model.parameters(), lr=args['learning_rate'])
  print('start training...')
  start_time = time.time()
  for epoch in range(num_epochs):
    start_time = time.time()
    # train
    train_loss = train_acc = 0
    model.train()
    for i, batch in enumerate(train_data):
      model.zero_grad()

      words, word_mask, postags, heads, deprels, sentlens = batch
      loss, acc, _ = model(words, word_mask, postags, heads, deprels, sentlens)
      train_loss += loss.data.cpu().numpy()
      train_acc += acc.data.cpu().numpy()

      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), args['clip'])
      optimizer.step()

    elapsed_time = time.time() - start_time
    train_loss /= len(train_data)
    train_acc /= len(train_data)

    # eval
    model.eval()
    eval_loss = eval_acc = 0
    for i, batch in enumerate(eval_data):
      model.zero_grad()

      words, word_mask, postags, heads, deprels, sentlens = batch
      loss, acc, _ = model(words, word_mask, postags, heads, deprels, sentlens)
      eval_loss += loss.data.cpu().numpy()
      eval_acc += acc.data.cpu().numpy()

    eval_loss /= len(eval_data)
    eval_acc /= len(eval_data)

    log = '|  {}/{} epoch  |  train_loss:{:.5f} | train_acc:{:2.2f} |  eval_loss:{:.5f} | eval_acc:{:2.2f} | time: {:.2f}  |'.format(
        epoch, num_epochs, train_loss, train_acc * 100, eval_loss, eval_acc * 100, elapsed_time
    )

    print(log)
  return

In [219]:
train_model(args, model, data, data, 3)

start training...
|  0/3 epoch  |  train_loss:3.66341 | train_acc:23.93 |  eval_loss:3.66762 | eval_acc:30.60 | time: 0.46  |
|  1/3 epoch  |  train_loss:3.64233 | train_acc:27.86 |  eval_loss:3.65619 | eval_acc:33.93 | time: 0.45  |
|  2/3 epoch  |  train_loss:3.67362 | train_acc:18.21 |  eval_loss:3.64528 | eval_acc:33.93 | time: 0.46  |


# Predict Function

In [ ]:
def predict(sentence):
  #TODO: sentence func
  return